## Using ReAct Framework to analyse a stock.
 - Get the latest stock data from a data source.
 - Get the stock symbol, its financials
 - Get the news data headlines from the internet.

* We will use Llama3.1 to do this.

In [1]:
import json
import sys
import os
from typing import Optional

from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
# from llama_index.tools.google import GoogleSearchToolSpec
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core import PromptTemplate

# finance tool
import yfinance as yf
import datetime as dt

# get the prompts
sys.path.append(os.path.dirname(os.path.abspath("../stock_analyser_react.ipynb")))
from prompts.react_agent_prompts import LLAMAINDEX_REACT_BASE_PROMPT, STOCK_ANALYSER_PROMPT

In [2]:
llm = Ollama(model='llama3.1:latest', request_timeout=120.0, temperature=0.01)
Settings.llm = llm

In [3]:
stream_response = llm.stream_complete("What is the stock symbol for Bajaj Finance?")

for t in stream_response:
    print(t.delta, end="")


# Note: how the model is hallucinating

The stock symbol for Bajaj Finance is BFIN.

In [4]:
# define the tools


def multiply(a: int, b: int) -> int:
    """Multiplies two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.
    
    Returns:
        int: The sum of a and b
    """
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer
    
    Args:
        a (int): The first integer.
        b (int): The second integer.
    
    Returns:
        int: The sum of a and b
    """
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The quotient of a divided by b.
    """
    return a // b


# yahoo finance financials fetcher
def get_financial_statements(ticker: str) -> str:
    """Fetches the financial statements data from yahoo finance.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.

    Returns:
        str: A string containing the financial statement data in a formatted way.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[-1] > 3:
        balance_sheet = balance_sheet.iloc[:, :3]

    balance_sheet = balance_sheet.dropna(how="any")
    balance_sheet = "\n" + balance_sheet.to_string()

    return balance_sheet


# get stock info and recommendations summary
def get_stockinfo_recommendations(ticker: str) -> str:
    """Provides the detailed financial and general information about the stock ticker.
        Also provides a table for analyst recommendations for the ticker.

    Args:
        ticker (str): The ticker symbol of the company to fetch information for.
    
    Returns:
        str: A string containing the stock info and a summary of recommendations.
    """

    if "." in ticker:
        ticker = ticker.split(".")[0]

    ticker += ".NS"
    company = yf.Ticker(ticker)

    

    stock_info = company.info
    try:
        recommendations_summary = company.get_recommendations()
    except:
        recommendations_summary = ""

    include_info = ["industry", "sector", "longBuisnessSummary", "previousClose", "dividendRate", "dividentYield", "beta", "forwardPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "currency", "bookValue", "priceToBook", "earningsQuarterlyGrowth", "trailingEps", "forwardEps", "52WeekChange", "totalCashPerShare", "ebidta", "totabDebt", "totalCashPerShare", "debtToEquity", "revenuePerShare", "earningsGrowth", "revenueGrowth", "grossMargins", "ebidtaMargins", "operatingMargins", ]
    response = "#### Stock info:\n"

    for key, val in stock_info.items():
        if key in include_info:
            response += f"{key}: {val}\n"

    response += "#### Recommendations:\n"
    response += f"\n{recommendations_summary}"

    return response


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
sub_tool = FunctionTool.from_defaults(fn=subtract)
div_tool = FunctionTool.from_defaults(fn=divide)

# financial sheets
fin_statements = FunctionTool.from_defaults(fn=get_financial_statements)

# stock information and the analyst recommendations
stockinfo_recommendations = FunctionTool.from_defaults(fn=get_stockinfo_recommendations)

ddg_tools = DuckDuckGoSearchToolSpec().to_tool_list()
ddg_full_search_tool = ddg_tools[1]
ddg_instant_search_tool = ddg_tools[0]

In [5]:
ddg_instant_search_tool.metadata

ToolMetadata(description='duckduckgo_instant_search(query: str) -> List[Dict]\n\n        Make a query to DuckDuckGo api to receive an instant answer.\n\n        Args:\n            query (str): The query to be passed to DuckDuckGo.\n        ', name='duckduckgo_instant_search', fn_schema=<class 'llama_index.core.tools.utils.duckduckgo_instant_search'>, return_direct=False)

In [6]:
# Define a ReactAgent
agent = ReActAgent.from_tools(tools=[ddg_full_search_tool, ddg_instant_search_tool, fin_statements, stockinfo_recommendations],
                   llm=llm,
                   verbose=True,
                   max_iterations=25)


# have a look at the agent prompt
agent.get_prompts()

{'agent_worker:system_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['tool_desc', 'tool_names'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\nYou have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools to complete each subtask.\n\nYou have access to the following tools:\n{tool_desc}\n\n\n## Output Format\n\nPlease answer in the same language as the question and use the following format:\n\n```\nThought: The current language of the user is: (user\'s language). I need to use a tool to help me answer the question.\nAction: tool name (one of {tool_names}) if using a tool.\nAction Input: the i

In [7]:
# assign the prompt to the agent
stock_analyser_prompt = PromptTemplate(template=STOCK_ANALYSER_PROMPT)
agent.update_prompts(
    {'agent_worker:system_prompt': stock_analyser_prompt}
)

In [11]:
response = agent.chat("Analyse based on the news Adani ports")

> Running step a930848e-a5df-4452-a25b-0967af161013. Step input: Analyse based on the news Adani ports
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: duckduckgo_full_search
Action Input: {'query': 'Adani Ports recent news', 'region': 'in'}
Observation: [{'title': 'Latest News and Stories | APSEZ - Adani Ports & SEZ', 'href': 'https://www.adaniports.com/Newsroom', 'body': "Read the latest News and Stories about Adani Ports and SEZ Ltd . About Us Explore About Us. Board of Directors. Managing Director's Message. Chairman Message. ... Adani Ports raises FY24 cargo target to over 400 mt; handles 311 mt in April-December Jan 03, 2024 ..."}, {'title': "India's Adani Ports Q4 profit soars 76% on record cargo volumes", 'href': 'https://www.reuters.com/world/india/indias-adani-ports-posts-bigger-q4-profit-strong-cargo-volumes-2024-05-02/', 'body': "The Adani group company last month said, opens new tab more than a fourth of In

In [ ]:
response